# Unveiling Cassava's Secrets
* AI Hackathon for Accelerating African Agrigenomics
* Goal of these notebook is to we will leverage experimentally mapped cassava enhancers and InstaDeep’s AgroNT model to build predictors for enhancer regulatory activity in the cassava genome.
* I will use a 1D CNN model
* Credits: https://www.kaggle.com/code/yasufuminakama/moa-pytorch-nn-starter/notebook

## Getting started with the Data
* Make sure you are using GPU

Cassava (Manihot esculenta) is the main food staple in Ghana, crucial for sustenance and income for many families. It makes up 22% of Ghana's Agricultural Gross Domestic Product (AGDP) and with an impressive per capita consumption rate of 152 kg, it is also responsible for 30% of the average daily calorie intake (Acheampong et al., 2021). Cassava is a prime example of an ‘orphan crop’, i.e. crops which have been under-researched in comparison to worldwide cash crops such as maize or rice (Yaqoob et al., 2023). One of cassava’s most intriguing secrets is its resistance to drought (Yaqoob et al., 2023), which can help the world face the challenges of climate change and a growing population.


In changing environmental conditions, such as exposure to drought, an organism responds by activating specific genes. Key regulators of gene activity are parts of DNA called ‘enhancers’ - specific sections of DNA that act like power boosters to increase gene expression. Standard genomics methods struggle to identify enhancers accurately because their activity is highly dependent on the context around them and they can be very far from their target genes (Wang et al., 2020).

In [ ]:
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import os
import copy
import seaborn as sns

from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedGroupKFold

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import warnings
warnings.filterwarnings('ignore')

In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

In [ ]:
path = "/kaggle/input/indabax-protein-sequence/"
train = pd.read_csv(path + "Train(4).csv")
test = pd.read_csv(path + "Test(4).csv")
#load the embeddings from AgroNt that were provided
train_embeds = np.load(path + "Train_embeddings.npy")
test_embeds = np.load(path + "Test_embeddings.npy")

# Create a DataFrame with the embeddings
embeds_train = pd.DataFrame(train_embeds, columns=[f'embed_{i}' for i in range(train_embeds.shape[1])])

# Create a DataFrame with the test embeddings
embeds_test = pd.DataFrame(test_embeds, columns=[f'embed_{i}' for i in range(test_embeds.shape[1])])


# Concatenate the embeddings DataFrame with your data_df
train = pd.concat([train, embeds_train], axis=1)
# Concatenate the test embeddings DataFrame with your test_df
test= pd.concat([test, embeds_test], axis=1)
display(train.head(), test.head())

In [ ]:
feature_cols = test.columns[4:].to_list()
target_cols = ['Target']
DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 100
BATCH_SIZE = 128
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-5
NFOLDS = 12
EARLY_STOPPING_STEPS = 30
EARLY_STOP = False


num_targets=train['Target'].nunique()
hidden_size=4096*4

### GroupKFold

In [ ]:
gkf = StratifiedGroupKFold(n_splits=12)

for f, (t_idx, v_idx) in enumerate(gkf.split(train, train['Target'], groups = train['Chromosome'])):
    train.loc[v_idx, 'fold'] = int(f)

train['fold'] = train['fold'].astype(int)
train['fold'].head()

### Introducing PCA
* To reduce the dimensionality reduction and also trying to introduce some spatial relationship

In [ ]:
from sklearn.decomposition import PCA
pca = PCA()
X_new = pca.fit_transform(train[feature_cols])
explained_variance=pca.explained_variance_ratio_
explained_variance

In [ ]:
len(explained_variance)

In [ ]:
import matplotlib.pyplot as plt

# Assuming you have an array explained_variance with 1500 elements
# Slice it to get the top 100
top_100_explained_variance = explained_variance[:60]

with plt.style.context('dark_background'):
    plt.figure(figsize=(6, 4))

    plt.bar(range(60), top_100_explained_variance, alpha=0.5, align='center',
            label='individual explained variance')
    plt.ylabel('Explained variance ratio')
    plt.xlabel('Principal components')
    plt.legend(loc='best')
    plt.tight_layout()

plt.show()


In [ ]:

# Initialize the PCA model with the desired number of components (60 in this case)
pca = PCA(n_components=60)

# Fit the PCA model to your selected columns
pca.fit(train[feature_cols])

# Transform the original data into the new principal components
train_pca_result = pca.transform(train[feature_cols])
test_pca_result = pca.transform(test[feature_cols])

# Create new column names for the principal components, e.g., PC1, PC2, ..., PC60
pc_columns = [f'PC{i+1}' for i in range(60)]

# Create a new DataFrame containing the principal components and concatenate it with the original DataFrame
pca_train_df = pd.DataFrame(data=train_pca_result, columns=pc_columns)
pca_test_df = pd.DataFrame(data=test_pca_result, columns=pc_columns)
train = pd.concat([train, pca_train_df], axis=1)
test = pd.concat([test, pca_test_df], axis=1)
train.head()

### Dataset Classes

In [ ]:
class TrainDataset:
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float),
            'y' : torch.tensor(self.targets[idx, :], dtype=torch.float)            
        }
        return dct
    
class TestDataset:
    def __init__(self, features):
        self.features = features
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float)
        }
        return dct

### Model

#### Helper Functions

In [ ]:
import numpy as np

def train_fn(model, optimizer, scheduler, loss_fn, dataloader, device):
    model.train()
    final_loss = 0
    correct_preds = 0
    total_samples = 0
    
    for data in dataloader:
        optimizer.zero_grad()
        inputs, targets = data['x'].to(device), data['y'].to(device)
        
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        scheduler.step()  # Allows you to control your model's LR according to some pre-set schedule or based on performance improvements
        
        # Calculate accuracy
        predicted_labels = (outputs.sigmoid() > 0.5).float()
        correct_preds += (predicted_labels == targets).sum().item()
        total_samples += targets.size(0)
        
        final_loss += loss.item()
    
    accuracy = correct_preds / total_samples
    final_loss /= len(dataloader)
    
    return final_loss, accuracy

def valid_fn(model, loss_fn, dataloader, device):
    model.eval()
    final_loss = 0
    valid_preds = []
    correct_preds = 0
    total_samples = 0
    
    for data in dataloader:
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        
        # Calculate accuracy
        predicted_labels = (outputs.sigmoid() > 0.5).float()
        correct_preds += (predicted_labels == targets).sum().item()
        total_samples += targets.size(0)
        
        final_loss += loss.item()
        valid_preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    accuracy = correct_preds / total_samples
    final_loss /= len(dataloader)
    valid_preds = np.concatenate(valid_preds)
    
    return final_loss, accuracy, valid_preds

def inference_fn(model, dataloader, device):
    model.eval()
    preds = []
    
    for data in dataloader:
        inputs = data['x'].to(device)

        with torch.no_grad():
            outputs = model(inputs)
        
        preds.append(outputs.sigmoid().cpu().numpy())  # Removed the unnecessary .detach()
        
    preds = np.concatenate(preds)
    
    return preds
   

#### Model
* BatchNormalization: done between the layers of a NN instead of in the raw data. It serves to speed up training and use higher Learning rates making learning easier
* If only tuning the CNN params was as easy as GBDT :)

In [ ]:
class Model(nn.Module):
    def __init__(self, num_features, hidden_size):
        super(Model, self).__init__()
        
        # Define channel sizes
        channel_1 = 256
        channel_2 = 512
        channel_3 = 512

        # Calculate reshape and pooling sizes
        channel_1_reshape = int(hidden_size / channel_1)
        channel_pool_1 = int(hidden_size / channel_1 / 2)
        channel_pool_2 = int(hidden_size / channel_1 / 2 / 2) * channel_3

        # Store values as instance variables
        self.channel_1 = channel_1
        self.channel_2 = channel_2
        self.channel_3 = channel_3
        self.channel_1_reshape = channel_1_reshape
        self.channel_pool_1 = channel_pool_1
        self.channel_pool_2 = channel_pool_2

        # Layer 1: Batch normalization, dropout, and linear transformation
        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dropout1 = nn.Dropout(0.1)
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_size))

        # Layer 2: Convolutional layer with batch normalization and dropout
        self.batch_norm_c1 = nn.BatchNorm1d(channel_1)
        self.dropout_c1 = nn.Dropout(0.1)
        self.conv1 = nn.utils.weight_norm(nn.Conv1d(channel_1, channel_2, kernel_size=5, stride=1, padding=2, bias=False), dim=None)

        # Layer 3: Adaptive average pooling
        self.ave_pool_c1 = nn.AdaptiveAvgPool1d(output_size=channel_pool_1)

        # Layer 4: Convolutional layer with batch normalization and dropout
        self.batch_norm_c2 = nn.BatchNorm1d(channel_2)
        self.dropout_c2 = nn.Dropout(0.1)
        self.conv2 = nn.utils.weight_norm(nn.Conv1d(channel_2, channel_2, kernel_size=3, stride=1, padding=1, bias=True), dim=None)

        # Layer 5: Additional convolutional layer with batch normalization and dropout
        self.batch_norm_c2_1 = nn.BatchNorm1d(channel_2)
        self.dropout_c2_1 = nn.Dropout(0.3)
        self.conv2_1 = nn.utils.weight_norm(nn.Conv1d(channel_2, channel_2, kernel_size=3, stride=1, padding=1, bias=True), dim=None)

        # Layer 6: Additional convolutional layer with batch normalization and dropout
        self.batch_norm_c2_2 = nn.BatchNorm1d(channel_2)
        self.dropout_c2_2 = nn.Dropout(0.2)
        self.conv2_2 = nn.utils.weight_norm(nn.Conv1d(channel_2, channel_3, kernel_size=5, stride=1, padding=2, bias=True), dim=None)

        # Layer 7: Max pooling
        self.max_pool_c2 = nn.MaxPool1d(kernel_size=4, stride=2, padding=1)

        # Layer 8: Flatten
        self.flatten = nn.Flatten()

        # New Output Layer for Binary Classification
        self.output_layer = nn.utils.weight_norm(nn.Linear(channel_pool_2, 1))

    def forward(self, x):
        # Layer 1: Batch normalization, dropout, and linear transformation
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = F.celu(self.dense1(x), alpha=0.06)

        # Reshape
        x = x.reshape(x.shape[0], self.channel_1, self.channel_1_reshape)

        # Layer 2: Convolutional layer with batch normalization and dropout
        x = self.batch_norm_c1(x)
        x = self.dropout_c1(x)
        x = F.relu(self.conv1(x))

        # Layer 3: Adaptive average pooling
        x = self.ave_pool_c1(x)

        # Layer 4: Convolutional layer with batch normalization and dropout
        x = self.batch_norm_c2(x)
        x = self.dropout_c2(x)
        x = F.relu(self.conv2(x))
        x_s = x

        # Layer 5: Additional convolutional layer with batch normalization and dropout
        x = self.batch_norm_c2_1(x)
        x = self.dropout_c2_1(x)
        x = F.relu(self.conv2_1(x))

        # Layer 6: Additional convolutional layer with batch normalization and dropout
        x = self.batch_norm_c2_2(x)
        x = self.dropout_c2_2(x)
        x = F.relu(self.conv2_2(x))
        x = x * x_s

        # Layer 7: Max pooling
        x = self.max_pool_c2(x)

        # Layer 8: Flatten
        x = self.flatten(x)

        # New Output Layer with Sigmoid Activation for Binary Classification
        logits = self.output_layer(x)
        probabilities = torch.sigmoid(logits)

        return probabilities  # Return probability scores


### Setup / Prepare

In [ ]:
feature_cols = [col for col in test.columns if 'PC' in col]
num_features=len(feature_cols)

### Single Fold Training

In [ ]:
def run_training(fold, seed):
    seed_everything(seed)
    trn_idx = train[train['fold'] != fold].index
    val_idx = train[train['fold'] == fold].index
    
    train_df = train[train['fold'] != fold].reset_index(drop=True)
    valid_df = train[train['fold'] == fold].reset_index(drop=True)
    
    x_train, y_train  = train_df[feature_cols].values, train_df[target_cols].values
    x_valid, y_valid =  valid_df[feature_cols].values, valid_df[target_cols].values  
    
    train_dataset = TrainDataset(x_train, y_train)
    valid_dataset = TrainDataset(x_valid, y_valid)
    
    trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
            num_features = num_features,
            hidden_size = hidden_size
    )
    
    model.to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE, weight_decay = WEIGHT_DECAY)
    
    #schedulers
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.01, div_factor=1e3, max_lr=1e-2, epochs=EPOCHS, steps_per_epoch=len(trainloader))
    #scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,factor=0.1,patience=1,verbose=True)
    #scheduler = optim.lr_scheduler.CyclicLR(optimizer=optimizer, base_lr = 1e-4, max_lr=1e-2,step_size_up=100,cycle_momentum=False,mode="triangular2")
    
    loss_fn = nn.BCEWithLogitsLoss()
    early_stopping_steps = EARLY_STOPPING_STEPS
    early_step = 0
    
    oof = np.zeros((len(train), num_targets))
    best_loss = np.inf
    
    for epoch in range(EPOCHS):
        train_loss, train_accuracy = train_fn(model, optimizer, scheduler, loss_fn, trainloader, DEVICE)
        print(f"FOLD: {fold}, EPOCH: {epoch}, train_loss: {train_loss}, train_accuracy: {train_accuracy}")
        valid_loss,valid_accuracy, valid_preds = valid_fn(model, loss_fn, validloader, DEVICE)
        print(f"FOLD: {fold}, EPOCH: {epoch}, valid_loss: {valid_loss}, valid_accuracy: {valid_accuracy}")
        #scheduler.step(valid_loss) #delete
        
        if valid_loss < best_loss:
            
            best_loss = valid_loss
            oof[val_idx] = valid_preds
            torch.save(model.state_dict(), f"FOLD{fold}_.pth")
        
        elif(EARLY_STOP == True):
            
            early_step += 1
            if (early_step >= early_stopping_steps):
                break        
                
                
# <<<<<<<<<<<---------------------------------PREDICTION------------------------------------------------------------>>>>>>>>>>>>>>>>>>>>>>>>>>

    x_test = test[feature_cols].values
    testdataset = TestDataset(x_test)
    testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)

    model = Model(
        num_features=num_features,
        hidden_size=hidden_size,
    )

    model.load_state_dict(torch.load(f"FOLD{fold}_.pth"))
    model.to(DEVICE)

    predictions = np.zeros((len(test), num_targets))
    predictions = inference_fn(model, testloader, DEVICE)

    return oof, predictions
    
        
        
    

In [ ]:
def run_k_folds(NFOLDS, seed):
    oof = np.zeros((len(train), num_targets))
    predictions = np.zeros((len(test), num_targets))
    for fold in range(NFOLDS):
        oof_, pred_ = run_training(fold, seed)
        
        predictions += pred_ / NFOLDS
        oof += oof_
        
    return oof, predictions
    

In [ ]:
SEED = [0, 1, 2]
oof = np.zeros((len(train), num_targets))
predictions = np.zeros((len(test), num_targets))

for seed in SEED:
    print(f"Training for seed: {seed}")
    
    oof_, predictions_ = run_k_folds(NFOLDS, seed)
    oof += oof_ / len(SEED)
    predictions += predictions_ / len(SEED)
    
    print(f" Finished Training for seed: {seed}")
    print()
          



In [ ]:
len(predictions), test.shape, len(oof), train.shape

In [ ]:
train['preds'] = oof[:, 1]
test['preds'] = predictions[:, 1]


In [ ]:
test['preds'].describe()

In [ ]:
from sklearn.metrics import accuracy_score
train['Label'] = np.where(train['preds']> 0.53, 1, 0)
accuracy_score(train['Label'], train['Target'])

In [ ]:
test['Label'] = np.where(test['preds']> 0.53, 1,0)
test['Label'].value_counts()

In [ ]:
sub = test[["ID","Label"]]
sub.to_csv("cnn_approach.csv", index = False)